In [ ]:
%%capture
!pip3 install seaborn

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
plats = "/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/resultados_all_writtenReps_plats.csv"
jap = "/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/resultados_all_writtenReps_jap.csv"
gold="/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/matches_gold.csv"

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score

# Cargar los archivos CSV
df_plats = pd.read_csv(plats)
df_jap = pd.read_csv(jap)

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)

# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Concatenar 'prefLabel' y 'writtenRep' para generar las oraciones para los embeddings
messages_plats = df_plats["prefLabel"].astype(str).str.cat(df_plats["writtenRep"].astype(str), sep=" ")
messages_jap = df_jap["prefLabel"].astype(str).str.cat(df_jap["writtenRep"].astype(str), sep=" ")

# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats.tolist()).numpy()
embeddings_jap = embed(messages_jap.tolist()).numpy()

# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)

# Establecer un umbral de similitud
umbral = 0.75

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Métricas de evaluación
y_true = [1 if (plats_id, jap_id) in gold_pairs else 0 for plats_id, jap_id in detected_pairs]
y_pred = [1] * len(detected_pairs)  # Todos los matches detectados son positivos

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


Detected Matches: 2
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Plats ID: plats_a_la_carta_C5043 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.9403
Plats ID: plats_a_la_carta_C7092 - Jap ID: gastronomia_japonesa_C16 - Similitud: 0.8087


In [ ]:
# Imprimir el porcentaje de matches del gold-standard que han superado el umbral
matches_en_gold = len(gold_pairs)
matches_detectados_en_gold = len([pair for pair in detected_pairs if pair in gold_pairs])
porcentaje_matches = (matches_detectados_en_gold / matches_en_gold) * 100 if matches_en_gold > 0 else 0
print(f"Porcentaje de matches del gold-standard superados el umbral: {porcentaje_matches:.2f}%")

Porcentaje de matches del gold-standard superados el umbral: 0.00%


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score


# Cargar los archivos CSV
df_plats = pd.read_csv(plats)
df_jap = pd.read_csv(jap)

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)

# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Usar solo 'writtenRep' para generar las oraciones para los embeddings
messages_plats = df_plats["writtenRep"].astype(str).tolist()  # Convertir a lista
messages_jap = df_jap["writtenRep"].astype(str).tolist()  # Convertir a lista

# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats).numpy()
embeddings_jap = embed(messages_jap).numpy()

# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)

# Establecer un umbral de similitud
umbral = 0.7

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Crear listas para las métricas de evaluación
y_true = []
y_pred = []

# Llenar y_true y y_pred
for plats_id, jap_id in detected_pairs:
    y_true.append(1 if (plats_id, jap_id) in gold_pairs else 0)
    y_pred.append(1)

# Agregar pares no detectados como negativos
for plats_id, jap_id in gold_pairs:
    if (plats_id, jap_id) not in detected_pairs:
        y_true.append(1)  # verdadero positivo
        y_pred.append(0)   # falso negativo

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


Detected Matches: 6
Precision: 0.1667
Recall: 0.1111
F1 Score: 0.1333
Plats ID: plats_a_la_carta_C5043 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.9878
Plats ID: plats_a_la_carta_C6164 - Jap ID: gastronomia_japonesa_C17 - Similitud: 0.7003
Plats ID: plats_a_la_carta_C6528 - Jap ID: gastronomia_japonesa_C10 - Similitud: 0.7399
Plats ID: plats_a_la_carta_C6573 - Jap ID: gastronomia_japonesa_C17 - Similitud: 0.7769
Plats ID: plats_a_la_carta_C7092 - Jap ID: gastronomia_japonesa_C16 - Similitud: 0.8536
Plats ID: plats_a_la_carta_C7093 - Jap ID: gastronomia_japonesa_C16 - Similitud: 0.7405


plats_a_la_carta_C7091,Plats_a_la_carta._Peix_i_marisc,"tempura_(trossos_de_peix,_marisc_i_verdures_arrebossats_i_fregits)"
plats_a_la_carta_C7092,Plats_a_la_carta._Peix_i_marisc,tempura_de_llagostins
plats_a_la_carta_C7093,"Plats_a_la_carta._Verdures,_hortalisses_i_llegums",tempura_de_verdures

gastronomia_japonesa_C16,Gastronomia,tempura

plats_a_la_carta_C6164,Plats_a_la_carta._Carn,pollastre_teriyaki_amb_fideus
- En el GOLD STANDARD falta el concepto TERIYAKI
gastronomia_japonesa_C17,Gastronomia,teriyaki
plats_a_la_carta_C6573,Plats_a_la_carta._Peix_i_marisc,salmó_teriyaki


plats_a_la_carta_C6528,Plats_a_la_carta._Licors,sake
gastronomia_japonesa_C10,Begudes,sake



In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score

# Cargar los archivos CSV
df_plats = pd.read_csv(plats)
df_jap = pd.read_csv(jap)

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)

# Función para extraer el primer 'writtenRep' o 'preflabel' si no existe 'writtenRep'
def get_first_writtenrep_or_preflabel(row, writtenrep_col, preflabel_col):
    # Asegurarse de que writtenRep no es NaN
    if pd.notna(row[writtenrep_col]):
        # Asumir que los multiple 'writtenRep' están separados por algún delimitador (coma, punto y coma)
        return row[writtenrep_col].split(",")[0]  # Obtener el primer valor
    else:
        # Si no hay writtenRep, usar preflabel
        return row[preflabel_col]

# Aplicar la función a las columnas de ambos dataframes
df_plats['text'] = df_plats.apply(lambda row: get_first_writtenrep_or_preflabel(row, 'writtenRep', 'preflabel'), axis=1)
df_jap['text'] = df_jap.apply(lambda row: get_first_writtenrep_or_preflabel(row, 'writtenRep', 'preflabel'), axis=1)

# Usar solo la columna 'text' para generar las oraciones para los embeddings
messages_plats = df_plats["text"].astype(str).tolist()  # Convertir a lista
messages_jap = df_jap["text"].astype(str).tolist()  # Convertir a lista

# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats).numpy()
embeddings_jap = embed(messages_jap).numpy()

# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)

# Establecer un umbral de similitud
umbral = 0.6

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Crear listas para las métricas de evaluación
y_true = []
y_pred = []

# Llenar y_true y y_pred
for plats_id, jap_id in detected_pairs:
    y_true.append(1 if (plats_id, jap_id) in gold_pairs else 0)
    y_pred.append(1)

# Agregar pares no detectados como negativos
for plats_id, jap_id in gold_pairs:
    if (plats_id, jap_id) not in detected_pairs:
        y_true.append(1)  # verdadero positivo
        y_pred.append(0)   # falso negativo

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


Detected Matches: 23
Precision: 0.0870
Recall: 0.2222
F1 Score: 0.1250
Plats ID: plats_a_la_carta_C13 - Jap ID: gastronomia_japonesa_C13 - Similitud: 0.6147
Plats ID: plats_a_la_carta_C1312 - Jap ID: gastronomia_japonesa_C12 - Similitud: 0.6391
Plats ID: plats_a_la_carta_C1376 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.6303
Plats ID: plats_a_la_carta_C1377 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.6303
Plats ID: plats_a_la_carta_C1378 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.6303
Plats ID: plats_a_la_carta_C1379 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.6303
Plats ID: plats_a_la_carta_C1380 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.6157
Plats ID: plats_a_la_carta_C1581 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.6165
Plats ID: plats_a_la_carta_C1592 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.6165
Plats ID: plats_a_la_carta_C1704 - Jap ID: gastronomia_japonesa_C13 - Similitud: 0.6011
Plats ID: plats_a_la_carta_C1739 - Jap ID: gastronomia_jap

no sé si tendrá que ver me aquellas que reciben un 1.0 de similitud son las que el dominio es más específico, en este caso:
- mirin
- sake

que a diferencia del resto de conceptos de gastronomia japonesa, su subdominio es "begudes", en vez de "alimentació" o "gastronomia"

In [ ]:
#HAY QUE HACERLO SIN GUIONES

In [ ]:
# Imprimir el porcentaje de matches del gold-standard que han superado el umbral
matches_en_gold = len(gold_pairs)
matches_detectados_en_gold = len([pair for pair in detected_pairs if pair in gold_pairs])
porcentaje_matches = (matches_detectados_en_gold / matches_en_gold) * 100 if matches_en_gold > 0 else 0
print(f"Porcentaje de matches del gold-standard superados el umbral: {porcentaje_matches:.2f}%")

Porcentaje de matches del gold-standard superados el umbral: 22.22%


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score


# Cargar los archivos CSV
df_plats = pd.read_csv(plats)
df_jap = pd.read_csv(jap)


# Cargar el gold-standard
gold_standard = pd.read_csv(gold)


# Función para extraer el primer 'writtenRep' o 'preflabel' si no existe 'writtenRep'
def get_first_writtenrep_or_preflabel(row, writtenrep_col, preflabel_col):
    # Asegurarse de que writtenRep no es NaN
    if pd.notna(row[writtenrep_col]):
        # Eliminar guiones bajos en la writtenRep
        return row[writtenrep_col].replace("_", " ")
    else:
        # Si no hay writtenRep, usar prefLabel, quitando guiones bajos
        return row[preflabel_col].replace("_", " ")


# Función para crear la oración con el formato especificado
def create_sentence(row, writtenrep_col, preflabel_col):
    # Obtener el primer writtenRep o preflabel
    first_writtenrep = get_first_writtenrep_or_preflabel(row, writtenrep_col, preflabel_col)
    # Crear la oración según el formato: "writtenRep és un plat de preflabel"
    sentence = f"{first_writtenrep} és un plat de {row[preflabel_col].replace('_', ' ')}"
    return sentence


# Aplicar la función para generar las oraciones formateadas
df_plats['text'] = df_plats.apply(lambda row: create_sentence(row, 'writtenRep', 'prefLabel'), axis=1)
df_jap['text'] = df_jap.apply(lambda row: create_sentence(row, 'writtenRep', 'prefLabel'), axis=1)


# Usar solo la columna 'text' para generar las oraciones para los embeddings
messages_plats = df_plats["text"].astype(str).tolist()  # Convertir a lista
messages_jap = df_jap["text"].astype(str).tolist()  # Convertir a lista


# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats).numpy()
embeddings_jap = embed(messages_jap).numpy()


# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)


# Establecer un umbral de similitud
umbral = 0.75


# Crear una lista para almacenar los matches detectados
detected_matches = []


# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))


# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])


# Crear listas para las métricas de evaluación
y_true = []
y_pred = []


# Llenar y_true y y_pred
for plats_id, jap_id in detected_pairs:
    y_true.append(1 if (plats_id, jap_id) in gold_pairs else 0)
    y_pred.append(1)


# Agregar pares no detectados como negativos
for plats_id, jap_id in gold_pairs:
    if (plats_id, jap_id) not in detected_pairs:
        y_true.append(1)  # verdadero positivo
        y_pred.append(0)   # falso negativo


# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)


# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


Detected Matches: 14
Precision: 0.1429
Recall: 0.2222
F1 Score: 0.1739
Plats ID: plats_a_la_carta_C2056 - Jap ID: gastronomia_japonesa_C12 - Similitud: 0.7253
Plats ID: plats_a_la_carta_C2056 - Jap ID: gastronomia_japonesa_C15 - Similitud: 0.7092
Plats ID: plats_a_la_carta_C5043 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.9636
Plats ID: plats_a_la_carta_C5090 - Jap ID: gastronomia_japonesa_C7 - Similitud: 0.7084
Plats ID: plats_a_la_carta_C5091 - Jap ID: gastronomia_japonesa_C7 - Similitud: 0.7078
Plats ID: plats_a_la_carta_C5093 - Jap ID: gastronomia_japonesa_C7 - Similitud: 0.7202
Plats ID: plats_a_la_carta_C6164 - Jap ID: gastronomia_japonesa_C17 - Similitud: 0.7151
Plats ID: plats_a_la_carta_C6528 - Jap ID: gastronomia_japonesa_C10 - Similitud: 0.7305
Plats ID: plats_a_la_carta_C6573 - Jap ID: gastronomia_japonesa_C17 - Similitud: 0.7274
Plats ID: plats_a_la_carta_C6648 - Jap ID: gastronomia_japonesa_C12 - Similitud: 0.7321
Plats ID: plats_a_la_carta_C6649 - Jap ID: gastronomi

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score


# Cargar los archivos CSV
df_plats = pd.read_csv(plats)
df_jap = pd.read_csv(jap)


# Cargar el gold-standard
gold_standard = pd.read_csv(gold)


# Función para extraer el primer 'writtenRep' o 'preflabel' si no existe 'writtenRep'
def get_first_writtenrep_or_preflabel(row, writtenrep_col, preflabel_col):
    # Asegurarse de que writtenRep no es NaN
    if pd.notna(row[writtenrep_col]):
        # Eliminar guiones bajos en la writtenRep
        return row[writtenrep_col].replace("_", " ")
    else:
        # Si no hay writtenRep, usar prefLabel, quitando guiones bajos
        return row[preflabel_col].replace("_", " ")


# Función para crear la oración con el formato especificado
def create_sentence(row, writtenrep_col, preflabel_col):
    # Obtener el primer writtenRep o preflabel
    first_writtenrep = get_first_writtenrep_or_preflabel(row, writtenrep_col, preflabel_col)
    # Crear la oración según el formato: "writtenRep és un plat de preflabel"
    sentence = f"{first_writtenrep} és un plat de {row[preflabel_col].replace('_', ' ')}"
    return sentence


# Aplicar la función para generar las oraciones formateadas
df_plats['text'] = df_plats.apply(lambda row: create_sentence(row, 'writtenRep', 'prefLabel'), axis=1)
df_jap['text'] = df_jap.apply(lambda row: create_sentence(row, 'writtenRep', 'prefLabel'), axis=1)


# Usar solo la columna 'text' para generar las oraciones para los embeddings
messages_plats = df_plats["text"].astype(str).tolist()  # Convertir a lista
messages_jap = df_jap["text"].astype(str).tolist()  # Convertir a lista


# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats).numpy()
embeddings_jap = embed(messages_jap).numpy()


# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)


# Establecer un umbral de similitud
umbral = 0.75


# Crear una lista para almacenar los matches detectados
detected_matches = []


# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            plats_text = df_plats['text'].iloc[i]
            jap_text = df_jap['text'].iloc[j]
            # Almacenar ID, similitud y las oraciones correspondientes
            detected_matches.append((plats_id, jap_id, similarity, plats_text, jap_text))


# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim, plats_text, jap_text in detected_matches])


# Crear listas para las métricas de evaluación
y_true = []
y_pred = []


# Llenar y_true y y_pred
for plats_id, jap_id in detected_pairs:
    y_true.append(1 if (plats_id, jap_id) in gold_pairs else 0)
    y_pred.append(1)


# Agregar pares no detectados como negativos
for plats_id, jap_id in gold_pairs:
    if (plats_id, jap_id) not in detected_pairs:
        y_true.append(1)  # verdadero positivo
        y_pred.append(0)   # falso negativo


# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)


# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


# Imprimir los matches encontrados con similitud y las secuencias correspondientes
for plats_id, jap_id, similarity, plats_text, jap_text in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")
    print(f"Plats Seq: {plats_text}")
    print(f"Jap Seq: {jap_text}\n")


Detected Matches: 2
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Plats ID: plats_a_la_carta_C5043 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.9636
Plats Seq: mirin, Mirin, mirin, mirin, mirin, mirin és un plat de Plats a la carta. Licors
Jap Seq: mirin, mirin, mirin, mirin és un plat de Begudes

Plats ID: plats_a_la_carta_C7092 - Jap ID: gastronomia_japonesa_C16 - Similitud: 0.8593
Plats Seq: tempura de llagostins, Garnelentempura, tempura prawns, tempura de langostinos, tempura de crevettes, tempura di mazzancolle és un plat de Plats a la carta. Peix i marisc
Jap Seq: tempura, tempura, tenpura, tempura, tenpura, tempoura, tempura és un plat de Gastronomia



In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score


# Cargar los archivos CSV
df_plats = pd.read_csv(plats)
df_jap = pd.read_csv(jap)


# Cargar el gold-standard
gold_standard = pd.read_csv(gold)


# Función para extraer solo la primera instancia de 'writtenRep' o 'prefLabel' si no existe 'writtenRep'
def get_first_writtenrep(row, writtenrep_col):
    # Dividir el valor por comas para obtener la primera instancia
    writtenrep_value = row[writtenrep_col].split(",")[0] if pd.notna(row[writtenrep_col]) else None
    if writtenrep_value:
        # Eliminar guiones bajos y devolver el primer valor
        return writtenrep_value.replace("_", " ")
    return None  # Retorna None si no existe


# Función para crear la oración solo con 'writtenRep'
def create_sentence(row, writtenrep_col):
    # Obtener solo la primera instancia de writtenRep
    first_writtenrep = get_first_writtenrep(row, writtenrep_col)
    # Crear la oración solo con la writtenRep
    return f"{first_writtenrep} és un plat" if first_writtenrep else None


# Aplicar la función para generar las oraciones formateadas
df_plats['text'] = df_plats.apply(lambda row: create_sentence(row, 'writtenRep'), axis=1)
df_jap['text'] = df_jap.apply(lambda row: create_sentence(row, 'writtenRep'), axis=1)


# Usar solo la columna 'text' para generar las oraciones para los embeddings
messages_plats = df_plats["text"].astype(str).tolist()  # Convertir a lista
messages_jap = df_jap["text"].astype(str).tolist()  # Convertir a lista


# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats).numpy()
embeddings_jap = embed(messages_jap).numpy()


# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)


# Establecer un umbral de similitud
umbral = 0.75


# Crear una lista para almacenar los matches detectados
detected_matches = []


# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            plats_text = df_plats['text'].iloc[i]
            jap_text = df_jap['text'].iloc[j]
            # Almacenar ID, similitud y las oraciones correspondientes
            detected_matches.append((plats_id, jap_id, similarity, plats_text, jap_text))


# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim, plats_text, jap_text in detected_matches])


# Crear listas para las métricas de evaluación
y_true = []
y_pred = []


# Llenar y_true y y_pred
for plats_id, jap_id in detected_pairs:
    y_true.append(1 if (plats_id, jap_id) in gold_pairs else 0)
    y_pred.append(1)


# Agregar pares no detectados como negativos
for plats_id, jap_id in gold_pairs:
    if (plats_id, jap_id) not in detected_pairs:
        y_true.append(1)  # verdadero positivo
        y_pred.append(0)   # falso negativo


# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)


# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


# Imprimir los matches encontrados con similitud y las secuencias correspondientes
for plats_id, jap_id, similarity, plats_text, jap_text in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")
    print(f"Plats Seq: {plats_text}")
    print(f"Jap Seq: {jap_text}\n")


Streaming output truncated to the last 5000 lines.
Plats ID: plats_a_la_carta_C6652 - Jap ID: gastronomia_japonesa_C14 - Similitud: 0.7653
Plats Seq: scamorza és un plat
Jap Seq: sukiyaki és un plat

Plats ID: plats_a_la_carta_C6652 - Jap ID: gastronomia_japonesa_C18 - Similitud: 0.8391
Plats Seq: scamorza és un plat
Jap Seq: wakame és un plat

Plats ID: plats_a_la_carta_C6652 - Jap ID: gastronomia_japonesa_C19 - Similitud: 0.8152
Plats Seq: scamorza és un plat
Jap Seq: xiitake és un plat

Plats ID: plats_a_la_carta_C6652 - Jap ID: gastronomia_japonesa_C20 - Similitud: 0.8156
Plats Seq: scamorza és un plat
Jap Seq: yakitori és un plat

Plats ID: plats_a_la_carta_C6652 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.8274
Plats Seq: scamorza és un plat
Jap Seq: kombu és un plat

Plats ID: plats_a_la_carta_C6652 - Jap ID: gastronomia_japonesa_C4 - Similitud: 0.8249
Plats Seq: scamorza és un plat
Jap Seq: matsutake és un plat

Plats ID: plats_a_la_carta_C6652 - Jap ID: gastronomia_japones

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score


# Cargar los archivos CSV
df_plats = pd.read_csv(plats)
df_jap = pd.read_csv(jap)


# Cargar el gold-standard
gold_standard = pd.read_csv(gold)


# Función para extraer solo la primera instancia de 'writtenRep'
def get_first_writtenrep(row, writtenrep_col):
    # Dividir el valor por comas para obtener la primera instancia
    writtenrep_value = row[writtenrep_col].split(",")[0] if pd.notna(row[writtenrep_col]) else None
    if writtenrep_value:
        # Eliminar guiones bajos y devolver el primer valor
        return writtenrep_value.replace("_", " ")
    return None  # Retorna None si no existe


# Función para crear la oración específica para cada DataFrame
def create_sentence_plats(row, writtenrep_col):
    # Obtener solo la primera instancia de writtenRep
    first_writtenrep = get_first_writtenrep(row, writtenrep_col)
    # Crear la oración para Plats a la Carta
    return f"{first_writtenrep} és un plat a la carta" if first_writtenrep else None

def create_sentence_jap(row, writtenrep_col):
    # Obtener solo la primera instancia de writtenRep
    first_writtenrep = get_first_writtenrep(row, writtenrep_col)
    # Crear la oración para Gastronomía Japonesa sin determinante
    return f"{first_writtenrep} és gastronomia japonesa" if first_writtenrep else None


# Aplicar la función para generar las oraciones formateadas
df_plats['text'] = df_plats.apply(lambda row: create_sentence_plats(row, 'writtenRep'), axis=1)
df_jap['text'] = df_jap.apply(lambda row: create_sentence_jap(row, 'writtenRep'), axis=1)


# Usar solo la columna 'text' para generar las oraciones para los embeddings
messages_plats = df_plats["text"].astype(str).tolist()  # Convertir a lista
messages_jap = df_jap["text"].astype(str).tolist()  # Convertir a lista


# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats).numpy()
embeddings_jap = embed(messages_jap).numpy()


# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)


# Establecer un umbral de similitud
umbral = 0.6


# Crear una lista para almacenar los matches detectados
detected_matches = []


# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            plats_text = df_plats['text'].iloc[i]
            jap_text = df_jap['text'].iloc[j]
            # Almacenar ID, similitud y las oraciones correspondientes
            detected_matches.append((plats_id, jap_id, similarity, plats_text, jap_text))


# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim, plats_text, jap_text in detected_matches])


# Crear listas para las métricas de evaluación
y_true = []
y_pred = []


# Llenar y_true y y_pred
for plats_id, jap_id in detected_pairs:
    y_true.append(1 if (plats_id, jap_id) in gold_pairs else 0)
    y_pred.append(1)


# Agregar pares no detectados como negativos
for plats_id, jap_id in gold_pairs:
    if (plats_id, jap_id) not in detected_pairs:
        y_true.append(1)  # verdadero positivo
        y_pred.append(0)   # falso negativo


# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)


# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


# Imprimir los matches encontrados con similitud y las secuencias correspondientes
for plats_id, jap_id, similarity, plats_text, jap_text in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")
    print(f"Plats Seq: {plats_text}")
    print(f"Jap Seq: {jap_text}\n")


Detected Matches: 1
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Plats ID: plats_a_la_carta_C3893 - Jap ID: gastronomia_japonesa_C11 - Similitud: 0.6054
Plats Seq: foie-gras semicuit a la sal és un plat a la carta
Jap Seq: sal de sèsam és gastronomia japonesa



Uso de Frases Estáticas: Cuando se fuerza un contexto específico en lugar de permitir que el modelo use frases más diversas, puede haber una reducción en la variedad léxica. Esto puede resultar en embeddings que son menos diferenciados y, por lo tanto, tienen menos probabilidades de mostrar similitudes significativas.
Repetición de Frases: Si la estructura es demasiado rígida, puede hacer que muchos términos se vuelvan muy similares entre sí, dificultando la identificación de distinciones importantes.

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score


# Cargar los archivos CSV
df_plats = pd.read_csv(plats)
df_jap = pd.read_csv(jap)


# Cargar el gold-standard
gold_standard = pd.read_csv(gold)


# Función para extraer solo la primera instancia de 'writtenRep' o 'prefLabel' si no existe 'writtenRep'
def get_first_writtenrep_or_preflabel(row, writtenrep_col, preflabel_col):
    # Dividir el valor por comas para obtener la primera instancia
    writtenrep_value = row[writtenrep_col].split(",")[0] if pd.notna(row[writtenrep_col]) else None
    if writtenrep_value:
        # Eliminar guiones bajos y devolver el primer valor
        return writtenrep_value.replace("_", " ")
    else:
        # Si no existe writtenRep, usar prefLabel eliminando guiones bajos
        return row[preflabel_col].split(",")[0].replace("_", " ")


# Función para crear la oración con el formato especificado
def create_sentence(row, writtenrep_col, preflabel_col):
    # Obtener solo la primera instancia de writtenRep o preflabel
    first_writtenrep = get_first_writtenrep_or_preflabel(row, writtenrep_col, preflabel_col)
    # Crear la oración según el formato: "writtenRep és un plat de preflabel"
    sentence = f"{first_writtenrep} és un plat de {row[preflabel_col].split(',')[0].replace('_', ' ')}"
    return sentence


# Aplicar la función para generar las oraciones formateadas
df_plats['text'] = df_plats.apply(lambda row: create_sentence(row, 'writtenRep', 'prefLabel'), axis=1)
df_jap['text'] = df_jap.apply(lambda row: create_sentence(row, 'writtenRep', 'prefLabel'), axis=1)


# Usar solo la columna 'text' para generar las oraciones para los embeddings
messages_plats = df_plats["text"].astype(str).tolist()  # Convertir a lista
messages_jap = df_jap["text"].astype(str).tolist()  # Convertir a lista


# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats).numpy()
embeddings_jap = embed(messages_jap).numpy()


# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)


# Establecer un umbral de similitud
umbral = 0.7


# Crear una lista para almacenar los matches detectados
detected_matches = []


# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            plats_text = df_plats['text'].iloc[i]
            jap_text = df_jap['text'].iloc[j]
            # Almacenar ID, similitud y las oraciones correspondientes
            detected_matches.append((plats_id, jap_id, similarity, plats_text, jap_text))


# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim, plats_text, jap_text in detected_matches])


# Crear listas para las métricas de evaluación
y_true = []
y_pred = []


# Llenar y_true y y_pred
for plats_id, jap_id in detected_pairs:
    y_true.append(1 if (plats_id, jap_id) in gold_pairs else 0)
    y_pred.append(1)


# Agregar pares no detectados como negativos
for plats_id, jap_id in gold_pairs:
    if (plats_id, jap_id) not in detected_pairs:
        y_true.append(1)  # verdadero positivo
        y_pred.append(0)   # falso negativo


# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)


# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


# Imprimir los matches encontrados con similitud y las secuencias correspondientes
for plats_id, jap_id, similarity, plats_text, jap_text in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")
    print(f"Plats Seq: {plats_text}")
    print(f"Jap Seq: {jap_text}\n")


Detected Matches: 3
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Plats ID: plats_a_la_carta_C4706 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7045
Plats Seq: llonganissa és un plat de Plats a la carta. Plats combinats
Jap Seq: kombu és un plat de Alimentació

Plats ID: plats_a_la_carta_C4707 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7045
Plats Seq: llonganissa és un plat de Plats a la carta. Plats combinats
Jap Seq: kombu és un plat de Alimentació

Plats ID: plats_a_la_carta_C7093 - Jap ID: gastronomia_japonesa_C16 - Similitud: 0.7168
Plats Seq: tempura de verdures és un plat de Plats a la carta. Verdures
Jap Seq: tempura és un plat de Gastronomia

